In [ ]:
# Install packages
# pipenv --python 3.11.0 install scikit-learn scipy pandas matplotlib seaborn ipykernel pingouin fastapi pydantic streamlit uvicorn requests

In [1]:
# Libs para rodar o EDA
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
# Carregar os dados
df = pd.read_csv('datasets/salarios.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   tempo_na_empresa  1000 non-null   int64  
 1   nivel_na_empresa  1000 non-null   int64  
 2   salario_em_reais  1000 non-null   float64
dtypes: float64(1), int64(2)
memory usage: 23.6 KB


In [4]:
# Criar cópias do DF
df_eda = df.copy()
df_bucketing = df.copy()